In [1]:
import pandas as pd
from newspaper import Article
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import wordnet 
from collections import Counter,OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize,word_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import enchant
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet

In [2]:
# f = open("articles.txt","r")
# LOA = [line.rstrip() for line in f]
# df = pd.DataFrame(columns = ['Title','Text','URL','Cleaned_text'])
# for url in LOA: 
#     try:
#         news_article = Article(url, language="en")     
#         news_article.download() 
#         news_article.parse() 
#         news_article.nlp() 
#         df2={'Title':news_article.title,'Text':news_article.text,'URL':url}
#         df=df.append(df2,ignore_index=True)
    
#     except Exception:
#         print(url)

# df.to_csv("news articles/parsed_news_articles_fresh.csv",encoding="utf-8")
df = pd.read_csv("RELIABILITY/data/parsed_news_articles.csv",encoding='utf-8')
df['Text'] = df['Text'].astype(str)

In [3]:
def preprocessor(text):
    
    stop_words = stopwords.words('english')
    sentences=sent_tokenize(text)
    d = enchant.Dict("en_US")
    spcl= '[-,;@_!#$%^&*()<>?/\|}{~:''.+""]'
    c=[]
    for i in sentences:
        english_words=[]
        i = i.lower()
        i = re.sub(r'[^\x00-\x7F]+',' ', i)
        i = re.sub('[0-9]',' ',i)
        i = re.sub(spcl,' ',i)
        i = re.sub('\n\n',' ',i)
        i = re.sub('\s\s',' ',i)
        v = i.split(' ')
        v = [word for word in v if word not in stop_words]
        v = [i for i in v if i!='']
        [english_words.append(j) for j in v if d.check(j)]
        temp=' '.join(word for word in english_words)
        c.append(temp)
    c=' '.join(sent for sent in c)
    return c



In [4]:
cleanedArticles=[]

for i in df.index:
    text=df['Text'][i]
    
    c=preprocessor(text)
    cleanedArticles.append(c)

df['Cleaned_text']=cleanedArticles

In [16]:
LOW = ['legendary', 'best', 'great', 'acclaimed', 'iconic', 'visionary', 'outstanding', 'leading', 'celebrated', 'landmark', 'cutting-edge', 'innovative', 'extraordinary', 'brilliant', 'hit', 'famous', 'renowned', 'remarkable', 'prestigious', 'world-class', 'respected', 'notable','virtuoso', 'honorable', 'awesome', 'unique', 'pioneering', 'phenomena','cult', 'racist', 'perverted', 'sexist','homophobic', 'transphobic', 'misogynistic', 'sect', 'fundamentalist', 'heretic', 'extremist', 'denialist', 'terrorist', 'bigot', 'myth', 'controversial','supposed', 'apparent', 'purported', 'alleged', 'accused','notably', 'arguably', 'interestingly', 'essentially', 'actually', 'clearly', 'happily', 'sadly', 'tragically', 'aptly', 'fortunately', 'unfortunately', 'untimely']



In [17]:
synonyms = []
antonyms = []

for word in LOW:
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())

synonyms = list(set(synonyms))
antonyms = list(set(antonyms))

print(len(synonyms))
print(len(antonyms))

words_to_check = list(set(synonyms+LOW))

393
26


In [18]:
print(words_to_check)

['honourable', 'reckon', 'Laputan', 'comfortably', 'respectable', 'estimable', 'slap-up', 'purpose', 'notably', 'conjecture', 'C._H._Best', 'come_to', 'windy', 'genius', 'unspoilt', 'incriminate', 'wiz', 'visionary', 'lionise', 'slay', 'prima', 'hotshot', 'acclaimed', 'open_up', 'corking', 'score', 'unluckily', 'ostensible', 'trump', 'over-the-top', 'better', 'leave', 'luminary', 'prestigious', 'craze', 'conduce', 'prominent', 'speculate', 'arguably', 'contribute', 'charge', 'propose', 'alas', 'misrepresented', 'practiced', 'cracking', 'large', 'perverse', 'clap', 'Best', 'effective', 'considerably', 'intimately', 'hypothecate', 'understandably', 'right', 'hypothesize', 'great', 'religious_outcast', 'radical', 'impeach', 'religious_cult', 'tragically', 'awing', 'guess', 'take', 'unparalleled', 'beneficial', 'smashing', 'vitiate', 'antiblack', 'apparent', 'sinful', 'neat', 'misuse', 'basically', 'polish_off', 'champion', 'whiz', 'legendary', 'reprobate', 'undischarged', 'honor', 'pip', 

In [14]:
df["strongP"]=""
scores = []
for article in cleanedArticles:
    c=0
    
    for word in set(article.split(' ')):
        
        if(word in words_to_check):
            c=c+1
    try:
        scores.append(float(c)/len(set(article.split(' '))))
        
    except Exception:
        scores.append(0.0)

df["strongP"]=scores



In [15]:
df

,Unnamed: 0,Source,Topic Name,Link,Title,Text,Cleaned_text,strongP
0,0,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/pm-m...,"PM Modi, Amit Shah must engage in dialogue ove...",Download The Times of India News App for Lates...,download times news app latest news subscribe ...,0.014815
1,1,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/trum...,"Trump to discuss CAA, NRC issues with Modi dur...",United States President Donald Trump with Prim...,united states president trump prime minister f...,0.039370
2,2,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/pm-m...,"Sonia Gandhi: PM Modi, Amit Shah misled people...",Download The Times of India News App for Lates...,download times news app latest news subscribe ...,0.021898
3,3,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/i-to...,"I told PM Modi to withdraw CAA, NRC and NPR: M...",Download The Times of India News App for Lates...,download times news app latest news subscribe ...,0.000000
4,4,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/pm-m...,PM Modi's visit to Bangladesh: Simmering tensi...,Prime Minister Narendra Modi. (File photo)\n\n...,prime minister file photo download times news ...,0.022556
5,5,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/cong...,Congress prepares to target Modi govt over CAA...,File photo (for representation only)\n\nDownlo...,file photo representation download times news ...,0.043103
6,6,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/caa-...,"CAA, NPR, NRC not just issues of Muslims, says...",Download The Times of India News App for Lates...,download times news app latest news subscribe ...,0.034722
7,7,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/city/nagpu...,"‘Agnipariksha must for Modi, Shah on CAA, NRC’",Swaraj Foundation and Akanksha Foundation join...,swaraj foundation foundation jointly organized...,0.024540
8,8,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/cong...,"Congress, urban Naxals spreading rumours over ...","This story is from December 22, 2019\n\nPM Nar...",story pm photo download times news app latest ...,0.019231
9,9,The Times of India,caa-nrc,https://timesofindia.indiatimes.com/india/caa-...,‘CAA derailed Congress plan to hit at govt on ...,Members of All Assam Students Union take out a...,members students union take rally photo downlo...,0.042105
